## Visualizatoin final project

### Imports

In [6]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re
# Set the style for the plots
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'sans-serif' 


os.getcwd()
BASE_DIR = os.path.dirname(os.getcwd())
BASE_DIR = os.path.join(BASE_DIR, 'final_project')
BASE_DIR


'/Users/noamzelig/Desktop/Noam/Visualization_course/final_project'

### load crime_df dataframe from yearly police reports published on gov.il data store using API

In [16]:
crime_df_agg = None
crime_df = None
crime_df_agg = pd.read_csv('crime_raw_cleaned.csv') # Example filename
print("✔ Loaded aggregated crime data from 'crime_raw_cleaned.csv' with shape:", crime_df_agg.shape)
print("Columns in crime_df_agg:", crime_df_agg.columns.tolist())    
print("✔ Crime counts by year:")   
print(crime_df_agg.groupby('Year').agg({'EventCount': 'sum'}).reset_index())

crime_df= pd.read_csv('crime_raw.csv') # Example filename
print("Columns in crime_df:", crime_df.columns.tolist())

print("✔ Crime counts by year:")   
print(crime_df.groupby('Year').agg({'_id': 'size'}).reset_index())


if crime_df_agg is None:
    crime_df = load_crime_df()     
    crime_df.to_csv('crime_raw.csv', index=False, encoding='utf-8-sig')
    print("✔ Saved raw crime data to 'crime_raw.csv'")
    print("✔ Crime counts by year:")
    print(crime_df.groupby('Year').agg({'_id': 'size'}).reset_index())
    


def load_crime_df():
    # resource_ids by year from data.gov.il
    resources = {
        "2025": "e311b6a1-be5a-4a82-8298-f3afbee07b6b",
        "2024": "5fc13c50-b6f3-4712-b831-a75e0f91a17e",
        "2023": "32aacfc9-3524-4fba-a282-3af052380244",
        "2022": "a59f3e9e-a7fe-4375-97d0-76cea68382c1",
        "2021": "3f71fd16-25b8-4cfe-8661-e6199db3eb12",
        "2020": "520597e3-6003-4247-9634-0ae85434b971"
    }

    all_dfs = []

    for year, resource_id in resources.items():
        print(f"loading {year}...")
        
        # API CKAN – limit 50000, work with offset 
        url = f"https://data.gov.il/api/3/action/datastore_search"
        limit = 50000
        offset = 0
        rows = []
        
        while True:
            resp = requests.get(url, params={
                "resource_id": resource_id,
                "limit": limit,
                "offset": offset
            })
            data = resp.json()
            
            batch = data["result"]["records"]
            rows.extend(batch)
            
            if len(batch) < limit:
                break  # הגיעו לסוף
            offset += limit
        
        df = pd.DataFrame(rows)
        all_dfs.append(df)

    # Combine all years into a single DataFrame
    crime_df = pd.concat(all_dfs, ignore_index=True)
    crime_df = crime_df.drop_duplicates()

    print("✔ Loaded total:", crime_df.shape[0], "rows")
    print("Columns in crime_df:", crime_df.columns.tolist())
   
    #print(crime_df.groupby('Year').size().reset_index(name='EventCount'))   

    return crime_df

✔ Loaded aggregated crime data from 'crime_raw_cleaned.csv' with shape: (571740, 12)
Columns in crime_df_agg: ['Year', 'Quarter', 'QuarterYear', 'Yeshuv', 'YeshuvKod', 'PoliceDistrict', 'PoliceMerhav', 'municipalName', 'StatisticArea', 'StatisticGroup', 'StatisticType', 'EventCount']
✔ Crime counts by year:
   Year  EventCount
0  2020      391532
1  2021      208697
2  2022      414130
3  2023      200342
4  2024      197531
5  2025      156033
Columns in crime_df: ['_id', 'FictiveIDNumber', 'Year', 'Quarter', 'YeshuvKod', 'Yeshuv', 'PoliceDistrictKod', 'PoliceDistrict', 'PoliceMerhavKod', 'PoliceMerhav', 'PoliceStationKod', 'PoliceStation', 'municipalKod', 'municipalName', 'StatisticAreaKod', 'StatisticArea', 'StatisticGroupKod', 'StatisticGroup', 'StatisticTypeKod', 'StatisticType']
✔ Crime counts by year:
   Year     _id
0  2020  391532
1  2021  208697
2  2022  414130
3  2023  200342
4  2024  197531
5  2025  156033


### crime df preproccesing 

In [ ]:

def clean_dataframe_columns(df, columns_to_clean):
    """
    Cleans specified columns in a DataFrame by removing special characters
    in a language-agnostic way.
    """
    df_cleaned = df.copy()
    pattern = r'[^\w\s]|_'
    
    print(f"--- Cleaning Text Columns: {columns_to_clean} ---")
    
    for col in columns_to_clean:
        if col in df_cleaned.columns:
            df_cleaned[col] = df_cleaned[col].apply(
                lambda x: re.sub(pattern, '', str(x)).strip() if pd.notna(x) else x
            )
        else:
            print(f"Warning: Column '{col}' not found.")
    print("Done.\n")
    return df_cleaned


def impute_missing_names_from_codes(df, code_name_pairs):
    """
    Imputes missing values in Name columns using their corresponding Code columns.
    """
    df_out = df.copy()
    print("--- Code-Based Imputation Report ---")
    
    for code_col, name_col in code_name_pairs:
        if code_col not in df_out.columns or name_col not in df_out.columns:
            continue
            
        missing_before = df_out[name_col].isna().sum()
        if missing_before == 0: continue
            
        valid_rows = df_out.dropna(subset=[code_col, name_col])
        if valid_rows.empty:
            print(f"Column '{name_col}': No valid reference data found.")
            continue

        mapping_df = valid_rows[[code_col, name_col]].drop_duplicates(subset=[code_col])
        mapping_dict = dict(zip(mapping_df[code_col], mapping_df[name_col]))
        
        mask = df_out[name_col].isna() & df_out[code_col].notna()
        df_out.loc[mask, name_col] = df_out.loc[mask, code_col].map(mapping_dict)
        
        missing_after = df_out[name_col].isna().sum()
        print(f"Column '{name_col}': Imputed {missing_before - missing_after} values using '{code_col}'. Remaining: {missing_after}")

    print("-------------------------------------\n")
    return df_out

def impute_fields_from_station_text(df, station_col, target_cols):
    """
    Imputes specified target columns (Yeshuv, Merhav, District, etc.) by looking 
    for their known values inside the PoliceStation text.
    
    STRICT RULES:
    1. Ignores '' (empty strings) and 'מקום אחר' (Other) as valid source values.
    2. Prioritizes longer names (greedy match) to avoid partial substring errors.
    """
    df_out = df.copy()
    
    if station_col not in df_out.columns:
        return df_out
        
    print(f"--- Text-Based Imputation (Mining '{station_col}') ---")
    
    # Define strictly forbidden values for imputation sources
    FORBIDDEN_VALUES = {'', ' ', 'מקום אחר', 'other', 'Other', 'לא ידוע', 'nan', 'None'}

    for target_col in target_cols:
        if target_col not in df_out.columns:
            continue
            
        missing_before = df_out[target_col].isna().sum()
        if missing_before == 0:
            print(f"  - Column '{target_col}': No missing values. Skipping.")
            continue

        # 1. Get List of known Valid Values
        known_values = df_out[target_col].dropna().unique()
        
        # Filter out forbidden values
        valid_candidates = [
            v for v in known_values 
            if str(v).strip() not in FORBIDDEN_VALUES
        ]
        
        # Sort by length descending to match longest possible name first 
        # (e.g., match "Tel Aviv-Yafo" before "Tel Aviv")
        valid_candidates = sorted(valid_candidates, key=lambda x: len(str(x)), reverse=True)
        
        if not valid_candidates:
            print(f"  - Column '{target_col}': No valid candidates found for text mining.")
            continue

        # 2. Find rows with missing Target but present Station
        mask = df_out[target_col].isna() & df_out[station_col].notna()
        stations_to_check = df_out.loc[mask, station_col].unique()
        
        mapping = {}
        
        for station in stations_to_check:
            st_str = str(station)
            for candidate in valid_candidates:
                cand_str = str(candidate)
                # Check if the candidate name is inside the station string
                if cand_str in st_str:
                    mapping[station] = candidate
                    break # Stop after finding the longest match
        
        # 3. Apply
        if not mapping:
            print(f"  - Column '{target_col}': No text matches found.")
        else:
            rows_to_fill = mask & df_out[station_col].isin(mapping.keys())
            df_out.loc[rows_to_fill, target_col] = df_out.loc[rows_to_fill, station_col].map(mapping)
            
            filled_count = df_out.loc[mask, target_col].notna().sum()
            print(f"  - Column '{target_col}': Imputed {filled_count} values from Station text.")

    print("-----------------------------------------------------------\n")
    return df_out

def impute_parent_from_child(df, child_col, parent_col):
    """
    Imputes missing values in a 'Parent' column (higher hierarchy) based on 
    values in a 'Child' column (lower hierarchy).
    """
    df_out = df.copy()
    
    if child_col not in df_out.columns or parent_col not in df_out.columns:
        return df_out

    missing_before = df_out[parent_col].isna().sum()
    if missing_before == 0:
        return df_out

    # 1. Build Mapping
    valid_relations = df_out.dropna(subset=[child_col, parent_col])[[child_col, parent_col]].drop_duplicates()
    
    # Filter out empty/invalid parents from mapping
    valid_relations = valid_relations[
        ~valid_relations[parent_col].astype(str).isin(['', ' ', 'מקום אחר', 'nan'])
    ]

    # 2. Check for Ambiguity
    ambiguous_children = valid_relations[valid_relations.duplicated(subset=[child_col], keep=False)][child_col].unique()
    
    if len(ambiguous_children) > 0:
        valid_relations = valid_relations[~valid_relations[child_col].isin(ambiguous_children)]
    
    mapping_dict = dict(zip(valid_relations[child_col], valid_relations[parent_col]))
    
    # 3. Apply Imputation
    mask = df_out[parent_col].isna() & df_out[child_col].notna()
    children_triggering_impute = df_out.loc[mask, child_col].unique()
    used_map = {k: v for k, v in mapping_dict.items() if k in children_triggering_impute}
    
    df_out.loc[mask, parent_col] = df_out.loc[mask, child_col].map(mapping_dict)
    actual_filled = df_out.loc[mask, parent_col].notna().sum()
    
    print(f"Hierarchy Imputation ('{child_col}' -> '{parent_col}'):")
    print(f"  - Missing Before: {missing_before}")
    print(f"  - Imputed: {actual_filled}")
    
    return df_out


def get_manual_code_mapping():
    """
    Returns a dictionary of manual code assignments for cities 
    where the code might be missing or the name mismatch prevents automatic coding.
    Key: City Name (in Crime Data)
    Value: Yeshuv Code (Standard CBS Code)
    """
    return {
        "נהריה": 9100,
        "קרית גת": 2630,
        "גסר א זרקא": 541,
        "נמל תעופה בן גוריון": 1748,
    }

def inject_manual_codes(df, city_col='Yeshuv', code_col='YeshuvCode'):
    """
    Manually injects Yeshuv Codes for specific city names.
    
    Args:
        df (pd.DataFrame): The crime dataframe.
        city_col (str): The column containing city names.
        code_col (str): The column to store/overwrite the City Code.
    """
    mapping = get_manual_code_mapping()
    print(f"--- Injecting Manual Yeshuv Codes into '{code_col}' ---")

    # Ensure code column exists; if not, create it
    if code_col not in df.columns:
        print(f"Creating missing column: {code_col}")
        df[code_col] = np.nan

    count_updates = 0
    # Create a copy to avoid SettingWithCopyWarning
    df = df.copy()

    for city_name, code in mapping.items():
        # Mask: Rows with this city name
        mask = (df[city_col] == city_name)
        rows_affected = mask.sum()
        
        if rows_affected > 0:
            # We force the code for these rows
            df.loc[mask, code_col] = code
            print(f"Set code {code} for '{city_name}': {rows_affected} rows updated.")
            count_updates += rows_affected
            
    print(f"✔ Total manual code updates: {count_updates}\n")
    return df

def process_and_summarize_crime_data(df):
    """
    Main execution function to clean, impute (Code, Hierarchy, Text), and aggregate crime data.
    """
    # 0. Print Initial Counts
    print("--- Initial Data Counts (Before Processing) ---")
    print(f"Total Records: {len(df)}")
    if 'Year' in df.columns:
        print("Records by Year:")
        print(df['Year'].value_counts().sort_index())
    print("-------------------------------------------\n")

    impute_pairs = [
        ('municipalKod', 'municipalName'),
        ('YeshuvKod', 'Yeshuv'),
        ('PoliceDistrictKod', 'PoliceDistrict'),
        ('PoliceMerhavKod', 'PoliceMerhav'),
        ('StatisticAreaKod', 'StatisticArea'),
        ('StatisticGroupKod', 'StatisticGroup'),
        ('StatisticTypeKod', 'StatisticType')
    ]
    
    # 1. Clean Text Columns
    text_cols = [pair[1] for pair in impute_pairs] + ['PoliceStation'] 
    text_cols = [c for c in text_cols if c in df.columns]
    df_processed = clean_dataframe_columns(df, text_cols)
    #df_processed = inject_manual_codes(df_processed)

    
    # 2. Impute Names from Codes
    df_processed = impute_missing_names_from_codes(df_processed, impute_pairs)
    
    # 3. Impute Hierarchy (Bottom-Up)
    print("--- Hierarchical Imputation Sequence ---")
    
    # A. Strict Hierarchy: PoliceStation -> Yeshuv
    df_processed = impute_parent_from_child(df_processed, 'PoliceStation', 'Yeshuv')
    
    # B. Text Imputation: PoliceStation -> [Yeshuv, Merhav, District, Muni]
    # This checks if the Station Name actually contains the name of the missing field
    text_mining_targets = ['Yeshuv', 'PoliceMerhav', 'PoliceDistrict', 'municipalName']
    df_processed = impute_fields_from_station_text(df_processed, 'PoliceStation', text_mining_targets)
    
    # C. Upstream Hierarchy: Yeshuv -> Merhav -> District, Yeshuv -> Muni
    # Uses the Yeshuvs/Merhavs filled by steps A and B to propagate further up
    remaining_steps = [
        ('Yeshuv', 'PoliceMerhav'),        
        ('Yeshuv', 'municipalName'),       
        ('PoliceMerhav', 'PoliceDistrict') 
    ]
    
    for child, parent in remaining_steps:
        df_processed = impute_parent_from_child(df_processed, child, parent)
    print("-------------------------------------------\n")

    # 4. Add Date Column
    df_processed['QuarterYear']= df_processed['Quarter'] +'-'+ df_processed['Year'].astype(str)
    #df_processed.head(10000).to_csv(os.path.join(BASE_DIR, 'crime_data_processed.csv'), index=False)
    
    # 5. Aggregate
    group_cols = [
        'Year', 'Quarter', 'QuarterYear', 'Yeshuv', 'YeshuvKod',
        'PoliceDistrict', 'PoliceMerhav', 'municipalName', 
        'StatisticArea', 'StatisticGroup', 'StatisticType'
    ]
    
    valid_group_cols = [c for c in group_cols if c in df_processed.columns]
    
    print(f"Aggregating by: {valid_group_cols}...\n")
    
    df_for_agg = df_processed.copy()
    for col in valid_group_cols:
        df_for_agg[col] = df_for_agg[col].fillna('Missing')
    
    summary_df = df_for_agg.groupby(valid_group_cols)['FictiveIDNumber'].count().reset_index()
    summary_df.rename(columns={'FictiveIDNumber': 'EventCount'}, inplace=True)
    
    print("--- Aggregated Summary Stats (After Processing) ---")
    print(f"Total Events: {summary_df['EventCount'].sum()}")
    print("Events by Year (Summary):")
    print(summary_df.groupby('Year')['EventCount'].sum().sort_index())
    
    return summary_df,df_processed
    

      
crime_df_agg, crime_df_processed = process_and_summarize_crime_data(crime_df)



### load population data from gov.il data store using API

In [ ]:
import pandas as pd
import requests
import numpy as np
import os

# ==========================================
# HELPER FUNCTIONS
# ==========================================

def clean_dataframe_columns(df, columns):
    """
    Trims whitespace from specified string columns.
    """
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()
    return df

# ==========================================
# 1. FETCHING DATA (API)
# ==========================================

def fetch_population_data():
    """
    Fetches population data from data.gov.il API for specific years.
    Selects only relevant columns and renames them to English.
    """
    print("--- Starting Data Fetch ---")
    
    resources = {
        2019: '990ae78e-2dae-4a15-a13b-0b5dcc56056c', # Added 2019 for 2020 baseline calc
        2020: '2d218594-73e3-40de-b36b-23b22f0a2627',
        2021: '95435941-d7e5-46c6-876a-761a74a5928d',
        2022: '199b15db-3bcb-470e-ba03-73364737e352',
        2023: 'd47a54ff-87f0-44b3-b33a-f284c0c38e5a'
    }
    
    base_url = "https://data.gov.il/api/3/action/datastore_search"
    pop_dfs = []

    # Define the fields to keep and their Hebrew search patterns
    field_mapping = {
        'Yeshuv_Code': 'סמל יישוב',
        'Yeshuv_Name': 'שם יישוב',       
        'Religion_Code': 'דת יישוב',
        'Total_Population': 'סך הכל אוכלוסייה',
        'Total_Israelis': 'סך הכל ישראלים',
        'Jews_and_Others': 'יהודים ואחרים',
        'Arabs': 'ערבים'
    }

    for year, resource_id in resources.items():
        try:
            response = requests.get(base_url, params={'resource_id': resource_id, 'limit': 5000})
            response.raise_for_status()
            data = response.json()
            
            if data['success']:
                records = data['result']['records']
                df_temp = pd.DataFrame(records)
                
                found_cols = {}
                
                # 1. Yeshuv Name (Exclude 'English')
                name_col = next((c for c in df_temp.columns if 'שם יישוב' in c and 'אנגלית' not in c), None)
                if name_col:
                    found_cols['Yeshuv_Name'] = name_col
                
                # 2. Find other columns based on keywords
                for eng_key, heb_search in field_mapping.items():
                    if eng_key == 'Yeshuv_Name': continue 
                    match = next((c for c in df_temp.columns if heb_search in c), None)
                    if match:
                        found_cols[eng_key] = match
                
                if 'Yeshuv_Code' in found_cols or 'Yeshuv_Name' in found_cols:
                    rename_map = {v: k for k, v in found_cols.items()}
                    df_clean = df_temp[list(found_cols.values())].rename(columns=rename_map).copy()
                    df_clean['Year'] = year
                    pop_dfs.append(df_clean)
                    print(f"Fetched {year}: {len(df_clean)} rows")
                else:
                    print(f"Warning {year}: Critical columns missing.")
            else:
                print(f"API Error {year}: Success=False")
                
        except Exception as e:
            print(f"Connection/Data Error {year}: {e}")
            try:
                # Fallback purely for runtime robustness
                local_df = pd.read_csv('population_raw.csv')
                if year == 2023:
                    pop_dfs.append(local_df.rename(columns={
                        'סמל יישוב': 'Yeshuv_Code', 'שם יישוב': 'Yeshuv_Name', 
                        'דת יישוב': 'Religion_Code', 'סך הכל אוכלוסייה 2023 - ארעי': 'Total_Population',
                        'שנה': 'Year'
                    }))
            except:
                pass

    if not pop_dfs:
        print("No data fetched.")
        return pd.DataFrame()

    final_df = pd.concat(pop_dfs, ignore_index=True)
    return final_df

# ==========================================
# 2. PREPROCESSING
# ==========================================

def preprocess_population(df):
    print("\n--- Preprocessing Population Data ---")
    if df.empty: return df

    df['Yeshuv_Code'] = pd.to_numeric(df['Yeshuv_Code'], errors='coerce')
    df = clean_dataframe_columns(df, ['Yeshuv_Name'])
    
    numeric_cols = ['Total_Population', 'Total_Israelis', 'Jews_and_Others', 'Arabs', 'Religion_Code']
    for col in numeric_cols:
        if col in df.columns:
            if df[col].dtype == object:
                 df[col] = df[col].astype(str).str.replace(',', '')
            df[col] = pd.to_numeric(df[col], errors='coerce')

    print(f"Processed {len(df)} rows. Columns: {df.columns.tolist()}")
    return df

# ==========================================
# 3. QUARTERLY EXTRAPOLATION
# ==========================================

def extrapolate_quarters(pop_df, years_of_interest):
    print("\n--- Extrapolating Quarters ---")
    expanded_data = []
    
    valid_pop = pop_df.dropna(subset=['Yeshuv_Code']).copy()
    debug_count = 0
    
    for yeshuv_code, group in valid_pop.groupby('Yeshuv_Code'):
        group = group.sort_values('Year')
        yeshuv_name = group['Yeshuv_Name'].iloc[0]
        
        for year in years_of_interest:
            current_row = group[group['Year'] == year]
            if current_row.empty: continue
            
            curr_pop = current_row['Total_Population'].values[0]
            if pd.isna(curr_pop): curr_pop = 0
            
            # Optional attributes
            rel_code = current_row['Religion_Code'].values[0] if 'Religion_Code' in current_row else np.nan
            israelis = current_row['Total_Israelis'].values[0] if 'Total_Israelis' in current_row else np.nan
            jews = current_row['Jews_and_Others'].values[0] if 'Jews_and_Others' in current_row else np.nan
            arabs = current_row['Arabs'].values[0] if 'Arabs' in current_row else np.nan

            # Calculate Growth
            prev_row = group[group['Year'] == year - 1]
            growth_calculated = False

            if not prev_row.empty:
                prev_pop = prev_row['Total_Population'].values[0]
                if pd.notna(prev_pop):
                    diff = curr_pop - prev_pop
                    q_growth = diff / 4
                    quarters = [prev_pop + q_growth, prev_pop + (q_growth*2), prev_pop + (q_growth*3), curr_pop]
                    growth_calculated = True
                    
            if not growth_calculated:
                quarters = [curr_pop] * 4
            
            for q_idx, q_name in enumerate(['Q1', 'Q2', 'Q3', 'Q4']):
                val = quarters[q_idx]
                pop_val = int(val) if pd.notna(val) else 0

                expanded_data.append({
                    'Yeshuv_Code': yeshuv_code,
                    'Yeshuv_Name': yeshuv_name,
                    'Year': year,
                    'Quarter': q_name,
                    'Total_Population': pop_val, 
                    'Religion_Code': rel_code,
                    'Total_Israelis': israelis,
                    'Jews_and_Others': jews,
                    'Arabs': arabs
                })
                
    return pd.DataFrame(expanded_data)

# ==========================================
# 4. JOIN WITH CRIME DATA
# ==========================================

def join_crime_population(crime_df_agg, pop_quarterly_df):
    """
    Joins crime DataFrame with the extrapolated population data.
    Input:
        crime_df_agg: pandas DataFrame containing aggregated crime data.
        pop_quarterly_df: pandas DataFrame containing population data.
    """
    print("\n--- Starting Join Operation ---")
    
    # 1. Clean Crime Data before merge
    # Remove rows where crucial keys are missing in the crime data itself
    crime_df_clean = crime_df_agg.dropna(subset=['Yeshuv', 'Year']).copy()
    
    # Prepare Keys
    crime_df_clean['Join_Key_Code'] = pd.to_numeric(crime_df_clean['YeshuvKod'], errors='coerce')
    crime_df_clean['Join_Key_Name'] = crime_df_clean['Yeshuv'].astype(str).str.strip()
    
    total_crime_rows = len(crime_df_clean)
    
    # Columns to merge from population
    merge_cols = ['Yeshuv_Code', 'Year', 'Quarter', 'Total_Population', 'Religion_Code', 
                  'Total_Israelis', 'Jews_and_Others', 'Arabs']
    
    available_cols = [c for c in merge_cols if c in pop_quarterly_df.columns]
    
    # 2. Join by Code
    print("...Matching by Yeshuv Code")
    merged_df = pd.merge(
        crime_df_clean,
        pop_quarterly_df[available_cols],
        left_on=['Join_Key_Code', 'Year', 'Quarter'],
        right_on=['Yeshuv_Code', 'Year', 'Quarter'],
        how='left'
    )
    
    # 3. Join by Name (Fallback)
    print("...Matching by Yeshuv Name (Fallback)")
    name_merge_cols = ['Yeshuv_Name', 'Year', 'Quarter'] + [c for c in available_cols if c not in ['Yeshuv_Code', 'Year', 'Quarter']]
    
    merged_with_name = pd.merge(
        merged_df,
        pop_quarterly_df[name_merge_cols],
        left_on=['Join_Key_Name', 'Year', 'Quarter'],
        right_on=['Yeshuv_Name', 'Year', 'Quarter'],
        how='left',
        suffixes=('', '_NameFallback')
    )
    
    # Coalesce values
    target_fields = ['Total_Population', 'Religion_Code', 'Total_Israelis', 'Jews_and_Others', 'Arabs']
    for col in target_fields:
        fallback_col = f'{col}_NameFallback'
        if fallback_col in merged_with_name.columns:
             merged_with_name[col] = merged_with_name[col].fillna(merged_with_name[fallback_col])
    
    # Cleanup auxiliary columns
    cols_to_drop = [c for c in merged_with_name.columns if 'Join_Key' in c or '_NameFallback' in c]
    merged_final = merged_with_name.drop(columns=cols_to_drop, errors='ignore')

    # --- POST MERGE (Keeping Missing Population) ---
    print("...Analyzing match results (Keeping unmatched rows)")
    # Previously we dropped these: merged_final.dropna(subset=['Total_Population'])
    # Now we keep them.
    total_rows = len(merged_final)
    matched_rows = merged_final['Total_Population'].notna().sum()
    unmatched_rows = total_rows - matched_rows

    # --- COLUMN CONSOLIDATION ---
    print("...Consolidating duplicate ID columns")
    # 1. Names: Keep 'Yeshuv', fill from 'Yeshuv_Name' if missing, then drop 'Yeshuv_Name'
    if 'Yeshuv' in merged_final.columns and 'Yeshuv_Name' in merged_final.columns:
        merged_final['Yeshuv'] = merged_final['Yeshuv'].fillna(merged_final['Yeshuv_Name'])
        merged_final.drop(columns=['Yeshuv_Name'], inplace=True)

    # 2. Codes: Keep 'YeshuvKod', fill from 'Yeshuv_Code' if missing, then drop 'Yeshuv_Code'
    if 'YeshuvKod' in merged_final.columns and 'Yeshuv_Code' in merged_final.columns:
        merged_final['YeshuvKod'] = merged_final['YeshuvKod'].fillna(merged_final['Yeshuv_Code'])
        merged_final.drop(columns=['Yeshuv_Code'], inplace=True)

    # Stats
    print("\n=== Merge Stats ===")
    print(f"Total Output Rows: {total_rows}")
    print(f"Rows with Population Data: {matched_rows}")
    print(f"Rows missing Population Data: {unmatched_rows}")
    print("===================\n")

    return merged_final

# ==========================================
# MAIN EXECUTION
# ==========================================

if __name__ == "__main__":
    
    final_filename = 'merged_crime_population_final.csv'
    
    # --- CHECK FOR EXISTING FINAL FILE ---
    if os.path.exists(final_filename):
        print(f"✔ Found existing final dataset: '{final_filename}'. Loading directly...")
        final_df = pd.read_csv(final_filename)
        
        print("--- Dataset Statistics ---")
        print(final_df.info())
        
        # Reset pandas display format
        pd.reset_option('display.float_format')

        print("\nSample Data:")
        print(final_df[['Year', 'Yeshuv', 'Total_Population']].head())
        
    else:
        # --- 1. LOAD & CLEAN CRIME DATA ---
        print("1. Loading and Cleaning Crime Data...")
        # NOTE: In a real run, load your csv here. 
        # For this script to work standalone, we assume 'crime_df_agg' might exist in memory or load it.
        try:
            # Try loading from file if variable doesn't exist (simulated environment)
            if 'crime_df_agg' not in globals():
                crime_df_agg = pd.read_csv('crime_data_processed.csv') # Example filename
                
            initial_crime_count = len(crime_df_agg)
            
            # Step 1: Remove Duplicates
            crime_df_agg = crime_df_agg.drop_duplicates()
            print(f"   - Removed {initial_crime_count - len(crime_df_agg)} duplicates.")
            
            # Step 2: Save Raw Crime CSV
            crime_df_agg.to_csv('crime_raw_cleaned.csv', index=False, encoding='utf-8-sig')
            print("   - Saved 'crime_raw_cleaned.csv'")
            
        except Exception as e:
            print(f"Error loading crime data: {e}")
            crime_df_agg = None

        # --- 2. PROCESS POPULATION DATA ---
        print("\n2. Processing Population Data...")
        pop_processed_filename = 'population_processed.csv'
        pop_quarterly = pd.DataFrame()

        if os.path.exists(pop_processed_filename):
            print(f"   - Found '{pop_processed_filename}', loading directly...")
            pop_quarterly = pd.read_csv(pop_processed_filename)
        else:
            pop_raw = fetch_population_data()
            
            if not pop_raw.empty:
                pop_clean = preprocess_population(pop_raw)
                
                # Extrapolate
                pop_quarterly = extrapolate_quarters(pop_clean, years_of_interest=[2020, 2021, 2022, 2023])
                
                # Step 3: Save Population CSV
                pop_quarterly.to_csv(pop_processed_filename, index=False, encoding='utf-8-sig')
                print(f"   - Saved '{pop_processed_filename}'")

        if not pop_quarterly.empty:
            # --- 3. MERGE & STATS ---
            if crime_df_agg is not None:
                print("\n3. Merging Data...")
                
                # Step 4 & 5: Merge (now keeping missing)
                final_df = join_crime_population(crime_df_agg, pop_quarterly)
                
                # Step 6: Print Stats and Save
                print("--- Final Dataset Statistics ---")
                print(final_df.info())
                
                # Reset pandas display format
                pd.reset_option('display.float_format')

                print("\nSample Data:")
                print(final_df[['Year', 'Yeshuv', 'Total_Population']].head())
                
                # Step 7: Save to CSV
                print(f"\n--- Saving Final Data ---")
                final_df.to_csv(final_filename, index=False, encoding='utf-8-sig')
                print(f"✔ Saved final dataset to: {os.path.abspath(final_filename)}")
        else:
            print("⚠ Population data unavailable (Fetch failed and no local file).")

### load cost price index data from halamas data store using API

In [ ]:
def get_chained_quarterly_cpi(start_year, end_year):
    # 1. API Parameters
    cpi_id = 120010
    start_period = f"01-{start_year}"
    end_period = f"12-{end_year}"
    
    url = "https://api.cbs.gov.il/index/data/price"
    
    params = {
        "id": cpi_id,
        "startPeriod": start_period,
        "endPeriod": end_period,
        "format": "json",
        "download": "false"
    }

    print(f"Fetching data from CBS API for {start_year}-{end_year}...")
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Verify data structure exists
        if 'month' not in data or not data['month']:
            print("No data found.")
            return None

        # Navigate to the list of observations
        # Structure: data['month'][0] contains the series info
        # Inside that, 'date' is the list of monthly records
        series_data = data['month'][0] 
        observations = series_data.get('date', [])
        
        records = []
        for obs in observations:
            # Extract date components
            year = obs['year']
            month = obs['month']
            
            # Extract monthly percentage change
            # Note: The key in the JSON is simply 'percent'
            pct_change = obs.get('percent')
            
            if pct_change is not None:
                date_str = f"{year}-{month:02d}-01"
                records.append({
                    'date': date_str,
                    'monthly_percent_change': float(pct_change)
                })
            
        df = pd.DataFrame(records)
        
        # Convert to datetime and sort ascending (Crucial for chaining)
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date').reset_index(drop=True)
        
        # 2. Calculate Chained Index (Normalization)
        # We start with a base of 100.0 points at the beginning of the period
        df['chained_index'] = 100.0
        
        # Iterate and calculate cumulative index based on percentage changes
        # Formula: New_Index = Old_Index * (1 + percent_change / 100)
        # We skip the first row (it stays 100.0) and calculate from the second onwards
        for i in range(1, len(df)):
            prev_index = df.loc[i-1, 'chained_index']
            change_pct = df.loc[i, 'monthly_percent_change']
            
            new_index = prev_index * (1 + change_pct / 100)
            df.loc[i, 'chained_index'] = new_index

        # 3. Quarterly Aggregation
        # We calculate the MEAN of the chained index for the quarter
        # We also sum the percentage changes to see "Quarterly Inflation"
        quarterly_df = df.set_index('date').resample('Q').agg({
            'chained_index': 'mean',
            'monthly_percent_change': 'sum' # Sum of monthly changes approx equals quarterly inflation
        }).reset_index()
        
        # Formatting
        quarterly_df['quarter_name'] = quarterly_df['date'].dt.to_period('Q')
        quarterly_df = quarterly_df.rename(columns={
            'chained_index': 'avg_chained_index_points',
            'monthly_percent_change': 'total_quarterly_inflation_pct'
        })
        
        return quarterly_df[['quarter_name', 'avg_chained_index_points', 'total_quarterly_inflation_pct']]

    except Exception as e:
        print(f"Error: {e}")
        return None

# --- Main Execution ---
if __name__ == "__main__":
    cpi_df = get_chained_quarterly_cpi(2020, 2025)
    cpi_df.to_csv('quarterly_cpi_chained.csv', index=False, encoding='utf-8-sig')
    print("✔ Saved quarterly CPI data to 'quarterly_cpi_chained.csv'")
    
    if cpi_df is not None:
        print("\n--- Quarterly CPI (Chained, Base Jan 2020 = 100) ---")
        print(cpi_df)